# PAndas preprocessing

In [1]:
import pandas as pd

In [5]:
df =  pd.read_html('https://en.wikipedia.org/wiki/List_of_chapters_in_the_Quran')
df = df[0]

In [6]:
df.columns

Index(['#', 'Anglicized title(s)', 'Arabic title(s)', 'English title(s)',
       'Number of verses (Number of Rukūʿs)', 'Place of Revelation',
       'Egyptian Standard Chronological Order[6][7][8]',
       'Nöldeke's Chronological Order[6]', 'Muqatta'at (isolated letters)[4]',
       'Title refers to', 'Main theme(s)', 'Juz''],
      dtype='object')

In [7]:
df2 = df[['Anglicized title(s)', 'Number of verses (Number of Rukūʿs)']]

In [8]:
df2

,Anglicized title(s),Number of verses (Number of Rukūʿs)
0,Al-Fatihah,7 (1)
1,Al-Baqarah,286 (40)
2,Aali 'Imran,200 (20)
3,An-Nisa,176 (24)
4,Al-Ma'idah,120 (16)
...,...,...
110,Al-Masad,5 (1/3)
111,Al-Ikhlas,4 (1/3)
112,Al-Falaq,5 (1/3)
113,An-Naas,6 (1/3)


In [9]:
df2['Number of verses (Number of Rukūʿs)'] = df2['Number of verses (Number of Rukūʿs)'].str.split(' ', expand=True)[0]

C:\Users\Muhammad_Luqman\AppData\Local\Temp\ipykernel_4496\2766427244.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Number of verses (Number of Rukūʿs)'] = df2['Number of verses (Number of Rukūʿs)'].str.split(' ', expand=True)[0]


In [10]:
df2

,Anglicized title(s),Number of verses (Number of Rukūʿs)
0,Al-Fatihah,7
1,Al-Baqarah,286
2,Aali 'Imran,200
3,An-Nisa,176
4,Al-Ma'idah,120
...,...,...
110,Al-Masad,5
111,Al-Ikhlas,4
112,Al-Falaq,5
113,An-Naas,6


In [11]:
df2.to_csv('surah_aayah_count.csv', index=False)

# Scraping tests

In [7]:
import requests
import numpy as np
import pandas as pd
import os

In [3]:
surah_df = pd.read_csv('surah_aayah_count.csv').dropna()

def generate_dateset(imam):
    output_dir = 'data'
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    output_dir = os.path.join(output_dir, imam)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    for surah_index in range(0, 113):
        for aayah_number in range(1, int(surah_df.iloc[surah_index]['Number of verses (Number of Rukūʿs)'] + 1)):
            file_name = f'{str(surah_index+1).zfill(3)}{str(aayah_number).zfill(3)}'
            url = f"https://everyayah.com/data/{imam}/{file_name}.mp3"

            out_file_path = os.path.join(output_dir, f'{surah_index+1}_{aayah_number}.mp3')
            if os.path.exists(out_file_path):
                continue
            doc = requests.get(url)

            if doc.status_code == 200:
                print(f'{surah_index+1}_{aayah_number}.mp3')
                with open(out_file_path, 'wb') as f:
                    f.write(doc.content)
            else:
                raise Exception(f'Error: {doc.status_code} for {url}')

In [4]:
imam_df = pd.read_csv('audiocats.csv')

In [10]:
df_chunks = np.array_split(imam_df, 10)

In [11]:
df_chunks

[                                    imam
 0           Abdul_Basit_Mujawwad_128kbps
 1    AbdulSamad_64kbps_QuranExplorer.Com
 2           Abdul_Basit_Murattal_192kbps
 3            Abdul_Basit_Murattal_64kbps
 4  Abdullaah_3awwaad_Al-Juhaynee_128kbps
 5                Abdullah_Basfar_192kbps
 6                 Abdullah_Basfar_32kbps
 7                 Abdullah_Basfar_64kbps,
                                                imam
 8                          Abdullah_Matroud_128kbps
 9                    Abdurrahmaan_As-Sudais_192kbps
 10                    Abdurrahmaan_As-Sudais_64kbps
 11                    Abu_Bakr_Ash-Shaatree_128kbps
 12                     Abu_Bakr_Ash-Shaatree_64kbps
 13                              Ahmed_Neana_128kbps
 14    Ahmed_ibn_Ali_al-Ajamy_128kbps_ketaballah.net
 15  Ahmed_ibn_Ali_al-Ajamy_64kbps_QuranExplorer.Com,
                                         imam
 16                   Akram_AlAlaqimy_128kbps
 17                           Alafasy_128kbps
 18  

In [5]:
for imam in imam_df['Imam'].unique():
    generate_dateset(imam)

,imam
0,Abdul_Basit_Mujawwad_128kbps
1,AbdulSamad_64kbps_QuranExplorer.Com
2,Abdul_Basit_Murattal_192kbps
3,Abdul_Basit_Murattal_64kbps
4,Abdullaah_3awwaad_Al-Juhaynee_128kbps
...,...
75,translations/urdu_farhat_hashmi
76,translations/urdu_shamshad_ali_khan_46kbps
77,warsh/warsh_Abdul_Basit_128kbps
78,warsh/warsh_ibrahim_aldosary_128kbps
